In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs
from reversible2.invert import invert

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs

In [ ]:
import sklearn.datasets
def create_splitted_moon(n):
    X,y  = sklearn.datasets.make_moons(n*2, shuffle=False, noise=1e-4)
    train_inputs_a = np_to_var(X[0:n:2], dtype=np.float32)
    train_inputs_b = np_to_var(X[1:n:2], dtype=np.float32)
    return train_inputs_a, train_inputs_b

In [ ]:
X1, X2 = create_splitted_moon(n=10)

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(var_to_np(X1)[:,0], var_to_np(X1)[:,1])
plt.scatter(var_to_np(X2)[:,0], var_to_np(X2)[:,1])


In [ ]:
from reversible2.distribution import TwoClassDist

from reversible2.blocks import dense_add_block, conv_add_block_3x3
from reversible2.rfft import RFFT, Interleave
from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th
from reversible2.splitter import SubsampleSplitter


class ModelWithDist(nn.Module):
    def __init__(self, model, dist):
        super(ModelWithDist, self).__init__()
        self.model = model
        self.dist = dist
        
    def 
    
    
def create_model():
    feature_model = nn.Sequential(
        dense_add_block(2,200),
        dense_add_block(2,200),
        dense_add_block(2,200),
        dense_add_block(2,200),
    )
    class_dist = TwoClassDist(2,0, [0,1])
    model_with_dist = ModelWithDist(feature_model, class_dist)
    return model_with_dist

In [ ]:
n1 = create_model()
n2 = create_model()

In [ ]:
m1 = create_mixture(len(X1))
m2 = create_mixture(len(X2))

In [ ]:
# first train normally, see if test NLL problem appears/how strong etc.
# expectation: points are moved towards one line, points not seen are moved away
# -> NLL increases of not seen points
# take second network trained on not seen data
# expectation: symmetric effect 
# see if you evaluate kl divergence in second space, 

In [ ]:
for Xa,Xb,na,nb,ma,o_n,o_m in ((X1,X2,n1,n2,m1, op_n1,op_m1),
                          (X2,X1,n2,n1,m2,op_n2, op_m2)) 
    # Loss between prior and mixture,
    # Optimizing prior and model
    oa = na(Xa)
    mya = ma.sample(oa, detach=True)

    loss = -th.mean(na.log_probs(mya))
    o_n.zero_grad()
    loss.backward()
    o_n.step()

    # Loss between mixture and other prior, optimizing mixture
    with th.no_grad():
        oa = na(Xa)
    mya = ma.sample(oa, detach=False)
    mya_in_b = translate(mya, na,nb)
    loss = -th.mean(nb.log_probs(mya_in_b))
    o_m.zero_grad()
    loss.backward()
    o_m.step()



